#### This document was used to note down learnings from sources

In [2]:
import os
import gym
from stable_baselines3 import PPO 
from stable_baselines3.common.vec_env import DummyVecEnv #vectorize environment, train multiple agents at the same time
#used to evaluate policies
from stable_baselines3.common.evaluation import evaluate_policy
%load_ext tensorboard

Researching
Terminology + most popular models
https://towardsdatascience.com/an-overview-of-classic-reinforcement-learning-algorithms-part-1-f79c8b87e5af

Cart pole gym doc
https://www.gymlibrary.ml/environments/classic_control/cart_pole/

Control Task

Actor critic vs dqn
actor critic is a temporal differencec learning-> model free renforcement learning

The DQN (Deep Q-Network) algorithm was developed by DeepMind in 2015. It was able to solve a wide range of Atari games (some to superhuman level) by combining reinforcement learning and deep neural networks at scale.

https://www.tensorflow.org/agents/tutorials/0_intro_rl

https://www.ray.io/rllib speed up / parameter tuning

 Asynchronous Advantage Actor Critic (A3C)
 In the Actor-Critic method, the policy is referred to as the actor that proposes a set of possible actions given a state, and the estimated value function is referred to as the critic, which evaluates actions taken by the actor based on the given policy.

 Asynchronous: The algorithm is an asynchronous algorithm where multiple worker agents are trained in parallel, each with their own copy of the model and environment. This allows our algorithm to not only train faster as more workers are training in parallel, but also to attain a more diverse training experience as each workers’ experience is independent.
Advantage: Advantage is a metric to judge both how good its actions were, but also how they turned out. This allows the algorithm to focus on where the network’s predictions were lacking. Intuitively, this allows us to measure the advantage of taking action, a, over following the policy π at the given time step.
Actor-Critic: The Actor-Critic aspect of the algorithm uses an architecture that shares layers between the policy and value function.

 A2C is a synchronous, deterministic variant of Asynchronous Advantage Actor Critic (A3C) which we’ve found gives equal performance. ACKTR is a more sample-efficient reinforcement learning algorithm than TRPO and A2C, and requires only slightly more computation than A2C per update.

 https://openai.com/blog/baselines-acktr-a2c/
 

OpenAI Gym Spaces
Box - n dimensional tensor ,RANGE OF VALUES/ continuous value
Box(low,high,shape =(y,x))

Discrete- set of items, discrete numbers with specific mapping to something. Typically used for actions
Discrete(3) -> returns 0 ,1,2

Tuple- Combine spaces together 
Tuple((Discrete(2),Box(low,high,shape=())))

Dict- dictionary of spaces 

MultiBinary - one hot encoded set of binary values
One hot encoded vector of actions/spaces
MultiBinary(4)

MultiDiscrete- multiple sets of discrete values
MultiDiscrete([5,2,2])




In [3]:
env_name = "CartPole-v1"
env = gym.make(env_name)

Episode = one full game in the env

some env have fixed episode length 
cartpolev1 500 frames
others continous/play until out of lives

Testing out randomly running episodes

In [4]:
env.observation_space
#shows the observation space , its a box type

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [5]:
#test run

episodes = 100
for episode in range(1,episodes+1):
    state = env.reset() #reset get intial set of observations.
    #reset agent ,environment, observation, reward
    done = False
    score = 0

    while not done:
        env.render()
        #env.action_space is a Discrete(2) space
        action = env.action_space.sample() # get random discrete value 0 or 1
        n_state,reward,done,info = env.step(action) #pass through action 
        # returns next step of observation , positive or negative reward , if episode is done , additional info
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()


Episode:1 Score:18.0
Episode:2 Score:9.0
Episode:3 Score:24.0
Episode:4 Score:27.0
Episode:5 Score:11.0
Episode:6 Score:15.0
Episode:7 Score:21.0
Episode:8 Score:18.0
Episode:9 Score:10.0
Episode:10 Score:15.0
Episode:11 Score:32.0
Episode:12 Score:10.0
Episode:13 Score:36.0
Episode:14 Score:16.0
Episode:15 Score:22.0
Episode:16 Score:35.0
Episode:17 Score:11.0
Episode:18 Score:16.0
Episode:19 Score:12.0
Episode:20 Score:15.0
Episode:21 Score:20.0
Episode:22 Score:22.0
Episode:23 Score:36.0
Episode:24 Score:15.0
Episode:25 Score:19.0
Episode:26 Score:14.0
Episode:27 Score:14.0
Episode:28 Score:20.0
Episode:29 Score:14.0
Episode:30 Score:15.0
Episode:31 Score:10.0
Episode:32 Score:11.0
Episode:33 Score:24.0
Episode:34 Score:17.0
Episode:35 Score:20.0
Episode:36 Score:33.0
Episode:37 Score:27.0
Episode:38 Score:34.0
Episode:39 Score:17.0
Episode:40 Score:11.0
Episode:41 Score:46.0
Episode:42 Score:19.0
Episode:43 Score:9.0
Episode:44 Score:12.0
Episode:45 Score:32.0
Episode:46 Score:32.0

In [6]:
env.action_space

Discrete(2)

In [7]:
env.action_space.sample()

0

In [8]:
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [9]:
env.observation_space.sample()

array([-2.8629062e-01, -3.2566297e+38, -1.8597622e-01, -2.5095113e+38],
      dtype=float32)

Source : https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

Observation:          
Cart Position -4.8 to 4.8
Cart Velocity -inf to inf
Pole Angle    -.418 rad to .418rad
Pole Angular velocity  -inf to inf

Action
0 push left
1 push right

### Rewards
    Since the goal is to keep the pole upright for as long as possible, a reward of `+1` for every step taken,
    including the termination step, is allotted. The threshold for rewards is 475 for v1.

### Starting State
    All observations are assigned a uniformly random value in `(-0.05, 0.05)`
    ### Episode Termination
    The episode terminates if any one of the following occurs:
    1. Pole Angle is greater than ±12°
    2. Cart Position is greater than ±2.4 (center of the cart reaches the edge of the display)
    3. Episode length is greater than 500 (200 for v0)

Training
Model vs Model free RL
Model based (rl-lib)
Make prediction of the  future state of the model to generate best actions 
https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html

![MODELS](modeltypes.png)

Algo choice based on
Action Space
Discrete Single Process : DQN
Discrete Multi Processed: PPO / A2C
Continuous Single Process: SAC / TD3
Continuous Multi Processed: PPO/A2C






Different Algorithm, different training metrics

Evaluation metrics

Time metrics

Loss metrics

Other metrics


In [10]:
#make log directories
log_path = os.path.join('Training','Logs')
log_path

'Training\\Logs'

Training model  

In [11]:
env = gym.make(env_name)
env = DummyVecEnv([lambda: env])# wrapped env in a dummy vec env wrapper
#env creation function
model = PPO('MlpPolicy',env,verbose = 1,tensorboard_log=log_path)
 #multi-layer perceptron policy 

Using cpu device


In [12]:
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gym.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv, str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    target_kl: Union[float, NoneType] = None,
    tensorboard_log: Union[str, NoneType] = None,
    create_eval_env: bool = False,
    policy_kwargs: Union[Dict[str, Any], NoneType] = None,
    verbose: int = 0,
    seed: Union[int, NoneType] = None,
    device: Union[torch.device, str] = 'auto',
    _init_setup_model: bool = 

In [13]:
#more complex more timesteps
model.learn(total_timesteps = 20000)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 2842 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1664       |
|    iterations           | 2          |
|    time_elapsed         | 2          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00752403 |
|    clip_fraction        | 0.0851     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.687     |
|    explained_variance   | -0.00144   |
|    learning_rate        | 0.0003     |
|    loss                 | 8.71       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0126    |
|    value_loss           | 59.8       |
----------------------------------------
---------------------

In [14]:
PPO_Path  = os.path.join("Training","Saved Models","PPO_Model_Cartpole")

In [15]:
model.save(PPO_Path)

In [16]:
del model

In [17]:
model = PPO.load(PPO_Path, env = env)

In [18]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 2543 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1770        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005038779 |
|    clip_fraction        | 0.0528      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.543      |
|    explained_variance   | 0.711       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.224       |
|    n_updates            | 110         |
|    policy_gradient_loss | -0.00109    |
|    value_loss           | 7.14        |
-----------------------------------------
---

PPO doesn't show rollout metrics
ep_len_mean how many frames before dying
ep_rew_mean average rewards

In [19]:
!tensorboard --logdir=log_path

^C


Evaluation with evaluation function

In [21]:
#env = gym.wrappers.Monitor(env, "recording")
evaluate_policy(model,env, n_eval_episodes= 3,render=True)
# function goes through 10 different episodes/runs and evaluates the performance based on it

#model , env, how wmany episodes we ewant to evaluate, visualize it

c:\Users\Adrian_Lam\anaconda3\envs\baseline\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(500.0, 0.0)

average reward , standard deviation

In [22]:
env.close()

Testing model

In [23]:
#now using model trained
episodes = 10
for episode in range(1,episodes+1):
    obs = env.reset() #resetting observation instead
    done = False
    score = 0

    while not done:
        env.render()
        action,_ = model.predict(obs)  #model predicting based on observation
        obs, reward,done,info = env.step(action)  #making action based on models choice
        score += reward
    print('Episode:{} Score:{}'.format(episode,score))
env.close()

Episode:1 Score:[500.]
Episode:2 Score:[500.]
Episode:3 Score:[500.]
Episode:4 Score:[500.]
Episode:5 Score:[500.]
Episode:6 Score:[500.]
Episode:7 Score:[500.]
Episode:8 Score:[500.]
Episode:9 Score:[500.]
Episode:10 Score:[500.]


view logs tensorboard

In [24]:
training_log_path = os.path.join(log_path, 'PPO_3')
training_log_path 

'Training\\Logs\\PPO_3'

Key metrics to evaluate
1. Average reward
2. Average Episode length (how long agent is lasting,more important for env with no time limits)

Training Strategies (improvement)
1. Train Longer
2. Hyperparameter tuning 
3. Try Different Algos

Callbacks


In [27]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join("Training","Saved Models")
stop_callback = StopTrainingOnRewardThreshold(reward_threhold = 475 , verbose = 1)

eval_callback =EvalCallback(env,
 callback_on_new_best = stop_callback, #callback we want to run on the new best model 
 eval_freq=10000, #how frequently running eval_callback
 best_model_save_path=save_path,
 verbose =1)

In [ ]:
model = PPO('MlpPolicy', env,verbose =1  ,tensorboard_log=log_path
)


In [ ]:
model.learn(total_timesteps=20000,callback = eval_callback)





custom architectures format

In [1]:
#custom policy

net_arch = [dict(pi=[128,128,128,128], vf = [128,128,128,128])]

In [ ]:
model = PPO('MlpPolicy', env,verbose =1  ,tensorboard_log=log_path , policy_kwargs={'net_arch':net_arch}
)